# CD
### 1. Write a program to search for a given pattern in a set of files. It should support regular expressions. It should work similar to grep and fgrep of Linux environment. 


### main.c
```
#include <stdio.h>
int main(){
    int x = 0;
    if(x)
    x--;
    else
    ++x;
    return 0;
}
```

In [ ]:
import re

# s = input("Enter filename : ")
s = 'main.c'
f = open(s, 'r')
text = f.read()

symbols = ['!', '@', '#', '$', '%', '&', '^', '*']
oparators = ['+', '-', '*', '/', '=', '+=', '-=', '==', '<', '>', '<=', '>=']
keywords = ['auto','break', 'case', 'char', 'const', 'continue', 'default', 'do', 
			'double', 'else', 'enum', 'extern', 'float', 'for', 'goto', 'if', 
			'int', 'long', 'register', 'return', 'short', 'signed', 'sizeof', 'static', 
			'struct', 'switch', 'typedef', 'union', 'unsigned', 'void', 'volatile', 'while']
delimiters = [' ', '	', '.', ',', '\n', ';', '(', ')', '<', '>', '{', '}', '[', ']']


in_keywords = []
in_spl_symbols = []
in_oparators = []
in_delimiters = []
in_identifiers = []
in_constants = []

tokens = []
isStr = False
isWord = False
isCmt = 0
token = ''

for i in text:
	if i == '/':
		isCmt = isCmt+1

	elif isCmt == 2:
		if i == '\n':
			token = ''
			isCmt = 0
	
	elif i == '"' or i == "'":
		if isStr:
			tokens.append(token)
			token = ''
		isStr = not isStr 

	elif isStr:
		token = token+i
    
	elif i in symbols:
		tokens.append(i)
           
	elif i.isalnum() and not isWord:
		isWord = True
		token = i
    
	elif (i in delimiters) or (i in oparators):
		if token:
			tokens.append(token)
			token = ''
        
		if not (i==' ' or i=='\n' or i=='	'):
			tokens.append(i)

	elif isWord:
		token = token+i


for token in tokens:
	if token in symbols:
		in_spl_symbols.append(token)

	elif token in oparators:
		in_oparators.append(token)

	elif token in keywords:
		in_keywords.append(token)
				
	elif re.search("^[_a-zA-Z][_a-zA-Z0-9]*$",token):
		in_identifiers.append(token)
		
	elif token in delimiters:
		in_delimiters.append(token)
		
	else:
		in_constants.append(token)
	
						
print("No of tokens = ", len(tokens))   
print("\n\nNo. of keywords = ",len(in_keywords))
print("\nNo. of special symbols = ",len(in_spl_symbols))
print("\nNo. of oparators = ",len(in_oparators))
print("\nNo. of identifiers = ",len(in_identifiers))
print("\nNo. of constants = ",len(in_constants))
print("\nNo. of delimiters = ",len(in_delimiters))
f.close()   

No of tokens =  34

No. of keywords =  5
['int', 'int', 'if', 'else', 'return']

No. of special symbols =  1
['#']

No. of oparators =  7
['<', '>', '=', '-', '-', '+', '+']

No. of identifiers =  8
['include', 'stdio', 'h', 'main', 'x', 'x', 'x', 'x']

No. of constants =  2
['0', '0']

No. of delimiters =  11
['.', '(', ')', '{', ';', '(', ')', ';', ';', ';', '}']


### 2. Write programs to implement DFA and NFA.(Input : DFA or NFAand a string and Output : Verification of any given string for acceptance.) 

In [ ]:
class DFA:
    def __init__(self,Q,S,D,q0,F) -> None:
        self.Q = Q
        self.S = S # Sigma - Alphabets
        self.D = D # Delta 
        self.s = q0
        self.F = F
    
    def check(self,inp):
        if inp == '' and self.s == 0 :
            return 'ACCEPTED'
        state = self.s
        for i in inp:
            if i not in self.S:
                return 'Rejected - Not an Alphabet'
            state = self.D[(state,i)]
        if state in self.F:
            return 'ACCEPTED'
        return 'Rejected - Not in Language'

class NFA(DFA):

    def check(self,inp):
        if inp == '' and self.s == 0 :
            return 'ACCEPTED'
        state = [self.s]
        for i in inp:
            if i not in self.S:
                return 'Rejected - Not an Alphabet'
            newstate = []
            for j in state:
                if (j,i) in self.D:
                    tempstate = self.D[(j,i)]
                    for k in tempstate:
                        newstate.append(k)
            state = list(set(newstate))
        for s in state:
            if s in self.F:
                return 'ACCEPTED'
        return 'Rejected - Not in Language'


dfa = DFA(3,('a','b'),{(0,'a'):1,(0,'b'):0,(1,'a'):2,(1,'b'):1,(2,'a'):0,(2,'b'):2},0,{0})
dfa.check('aababaaab')

'ACCEPTED'

In [ ]:
dfa.check('aabbaa')

'Rejected - Not in Language'

In [ ]:
nfa = NFA(2,('a','b'),{(0,'a'):[1],(1,'b'):[0]},0,{1})
nfa.check('abababa')

'ACCEPTED'

In [ ]:
nfa.check('abab')

'Rejected - Not in Language'

### 3. Design a PDA for any given CNF. Simulate the processing of a string using the PDA and show the parse tree. 

In [ ]:
class DPDA:
    
    def __init__(self, trf, input, state):
        
        self.head = 0
        self.trf = {}
        self.state = str(state)
        self.input = input
        self.trf = trf
        self.stack = ['Z']
        
    def step(self):
        
        a = self.input[self.head]
        s = self.stack.pop()
        state, ss = self.trf.get((self.state, a, s))
        if ss != 'ε':
            for s in ss[::-1]:
                self.stack.append(s)
        self.state = state
        print('{:20s} [{:10s}] {:5s}'.format(self.input[self.head:], 
                       ''.join(self.stack), self.state))        
        self.head += 1
    
    def run(self):
        
        print('{:20s} [{:10s}] {:5s}'.format(self.input[self.head:], 
                              ''.join(self.stack), self.state))
        
        while self.head  < len(self.input):
            self.step()

        s = self.stack.pop()        
        if self.trf.get((self.state, 'ε', s)):
            state, ss = self.trf.get((self.state, 'ε', s))
            self.state = state        
            print('{:20s} [{:10s}] {:5s}'.format('ε', 
                 ''.join(self.stack), self.state))
        if self.state == 'acc':
            return "Accepted"
        return "Rejected"
        
# run DPDA to accept the input string a^9b^9
DPDA({('q', 'a', 'Z'): ('q', 'XZ'),
     ('q', 'a', 'X'): ('q', 'XX'),
     ('q', 'b', 'X'): ('p', 'ε'),
     ('p', 'b', 'X'): ('p', 'ε'),
     ('p', 'ε', 'Z'): ('acc', 'Z'),
    }, 
    'aaaaabbbbb', 'q').run()

aaaaabbbbb           [Z         ] q    
aaaaabbbbb           [ZX        ] q    
aaaabbbbb            [ZXX       ] q    
aaabbbbb             [ZXXX      ] q    
aabbbbb              [ZXXXX     ] q    
abbbbb               [ZXXXXX    ] q    
bbbbb                [ZXXXX     ] p    
bbbb                 [ZXXX      ] p    
bbb                  [ZXX       ] p    
bb                   [ZX        ] p    
b                    [Z         ] p    
ε                    [          ] acc  


'Accepted'

In [ ]:
Grammar = (('S','A','B'),('a','b'),{'S':['AB','BC'],'A':['BA','a'],'B':['CC','b'],'C':['AB','a']},'S') #VTPS

def CYK(Grammar,inp):
    n = len(inp)
    DP = [[[] for j in range(n)] for i in range(n)]
    for i in range(n):
        for k,v in Grammar[2].items():
            for r in v:
                if r == inp[i] :
                    DP[i][i].append(k)
    for i in range(2,n+1):
        for j in range(n-i+1):
            k = j+i-1
            for l in range(j,k-1):
                for prod,rules in Grammar[2].items():
                    for rule in rules:
                        if len(rule) > 1 and rule[0] == DP[j][l] and rule[1] == DP[l+1][k] :
                            DP[j][k].append(prod)
                            dis(DP,n)
def dis(DP,n):
    for i in range(n):
        for j in range(n):
            print(DP[i][j],end='\t\t')
        print()

    
CYK(Grammar , 'baaba')


In [ ]:
non_terminals = ["S", "N", "D", "P",
				"V", "A"]
terminals = ["a","b", "c", "d",
			"e", "f",
			"g", "h","i"]

# Rules of the grammar
R = {
	"S": [["D", "N"]],
	"N": [["P", "N"], ["b"],
			["c"], ["d"]],
	"P": [["V", "A"], ["f"],
			["c"], ["e"]],
	"D": [["a"]],
	"V": [["g"], ["i"]],
	"A": [["f"], ["c"], ["e"],
		["h"]]
	}

def cykParse(inp):
	n = len(inp)
	T = [[set([]) for j in range(n)] for i in range(n)]

	for j in range(0, n):
		for lhs, rule in R.items():
			for rhs in rule:
				if len(rhs) == 1 and rhs[0] == inp[j]:
					T[j][j].add(lhs)

		for i in range(j, -1, -1):
			for k in range(i, j + 1):	
				for lhs, rule in R.items():
					for rhs in rule:
						if len(rhs) == 2 and rhs[0] in T[i][k] and rhs[1] in T[k + 1][j]:
							T[i][j].add(lhs)

	if len(T[0][n-1]) != 0:
		print("True")
	else:
		print("False")

inp = "a g f c b".split()
cykParse(inp)


True


### 4. Design a Lexical analyzer for identifying different types of tokens used in C language. Note: The reserved keywords such as if, else, class, structetc must be reported as invalid identifiers. C allows identifier names to begin with underscore character too. 

In [ ]:
import re

# s = input("Enter filename : ")
s = 'main.c'
f = open(s, 'r')
text = f.read()

symbols = ['!', '@', '#', '$', '%', '&', '^', '*']
oparators = ['+', '-', '*', '/', '=', '+=', '-=', '==', '<', '>', '<=', '>=']
keywords = ['auto','break', 'case', 'char', 'const', 'continue', 'default', 'do', 
			'double', 'else', 'enum', 'extern', 'float', 'for', 'goto', 'if', 
			'int', 'long', 'register', 'return', 'short', 'signed', 'sizeof', 'static', 
			'struct', 'switch', 'typedef', 'union', 'unsigned', 'void', 'volatile', 'while']
delimiters = [' ', '	', '.', ',', '\n', ';', '(', ')', '<', '>', '{', '}', '[', ']']


in_keywords = []
in_spl_symbols = []
in_oparators = []
in_delimiters = []
in_identifiers = []
in_constants = []

tokens = []
isStr = False
isWord = False
isCmt = 0
token = ''

for i in text:
	if i == '/':
		isCmt = isCmt+1

	elif isCmt == 2:
		if i == '\n':
			token = ''
			isCmt = 0
	
	elif i == '"' or i == "'":
		if isStr:
			tokens.append(token)
			token = ''
		isStr = not isStr 

	elif isStr:
		token = token+i
    
	elif i in symbols:
		tokens.append(i)
           
	elif i.isalnum() and not isWord:
		isWord = True
		token = i
    
	elif (i in delimiters) or (i in oparators):
		if token:
			tokens.append(token)
			token = ''
        
		if not (i==' ' or i=='\n' or i=='	'):
			tokens.append(i)

	elif isWord:
		token = token+i


for token in tokens:
	if token in symbols:
		in_spl_symbols.append(token)

	elif token in oparators:
		in_oparators.append(token)

	elif token in keywords:
		in_keywords.append(token)
				
	elif re.search("^[_a-zA-Z][_a-zA-Z0-9]*$",token):
		in_identifiers.append(token)
		
	elif token in delimiters:
		in_delimiters.append(token)
		
	else:
		in_constants.append(token)
	
						
print("No of tokens = ", len(tokens))   
print("\nNo. of keywords = ",len(in_keywords))
print(in_keywords);
print("\nNo. of special symbols = ",len(in_spl_symbols))
print(in_spl_symbols);
print("\nNo. of oparators = ",len(in_oparators))
print(in_oparators);
print("\nNo. of identifiers = ",len(in_identifiers))
print(in_identifiers);
print("\nNo. of constants = ",len(in_constants))
print(in_constants);
print("\nNo. of delimiters = ",len(in_delimiters))
print(in_delimiters);
f.close()   

5. Program to recognize the identifiers, if and switch statements of C using a lexical analyzer generator tool. 

### 6. Consider the following grammar: 
#### S --> ABC 
#### A-->abA | ab 
#### B--> b | BC 
#### C--> c | cC 
### Design any shift reduced parser which accepts a string and tells whether the string is accepted by above grammar or not. 

In [ ]:
g = {'S': 'ABC', 'A': ['abA', 'ab'], 'B': ['b', 'BC'], 'C': ['c', 'cC']}
# G = (V,T,P,S)
G = (('S', 'A', 'B', 'C'), ('a', 'b', 'c', '$'), g, 'S')

Items = 11

pt_action = {
    (0, 'a'): 'S3',
    (1, '$'): 'ACC',
    (2, 'b'): 'S4',
    (3, 'b'): 'S6',
    (4, 'c'): 'R4',
    (5, 'c'): 'S8',
    (6, 'a'): 'S3',
    (6, 'b'): 'R3',
    (7, 'c'): 'R5',
    (7, '$'): 'R1',
    (8, 'c'): 'S8',
    (8, '$'): 'R6',
    (9, 'c'): 'R7',
    (9, '$'): 'R7',
    (10, 'b'): 'R2'
}
pt_goto = {
    (0, 'S'): 1,
    (0, 'A'): 2,
    (2, 'B'): 5,
    (5, 'C'): 7,
    (6, 'A'): 10,
    (8, 'C'): 9
}

R = {
    'R1': ['S', 'ABC'],
    'R2': ['A', 'abA'],
    'R3': ['A', 'ab'],
    'R4': ['B', 'b'],
    'R5': ['B', 'BC'],
    'R6': ['C', 'c'],
    'R7': ['C', 'cC']
}


def SRParser(s: str):
    s = s + '$'
    stack = ['$', 0]
    sptr = 0
    print('STACK\t\t\tISUT\t\t\tACTION')
    while(stack[-1] != 'ACC' or sptr < len(s)):
        if stack[-1] in range(Items):
            if s[sptr] in G[1]:
                if (stack[-1], s[sptr]) not in pt_action:
                    break
                act = pt_action[(stack[-1], s[sptr])]
                if 'S' in act:
                    stack.append(s[sptr])
                    stack.append(int(act[1]))
                    sptr += 1
                    print(f'{stack}    \t\t{s[sptr:]}    \t\tSHIFT {act}')
                elif 'R' in act:
                    red_move = R[act]
                    popnum = len(red_move[1]) * 2
                    for i in range(popnum):
                        stack.pop()
                    stack.append(red_move[0])
                    print(f'{stack}    \t\t{s[sptr:]}    \t\tREDUCE {act}')
                elif 'ACC' in act:
                    popnum = 2 * 2
                    for i in range(popnum):
                        stack.pop()
                    stack.append('ACC')
                    print(f'{stack}    \t\t{s[sptr:]}    \t\tACCEPT {act}')
                else:
                    break
        elif stack[-1] in G[0]:
            stack.append(pt_goto[(stack[-2], stack[-1])])
            print(f'{stack}    \t\t{s[sptr:]}    \t\tGOTO {stack[-1]}')
        else:
            break
    if stack[0] == 'ACC':
        return "Accepted"
    return "Rejected"


SRParser('abbc'),SRParser('abbbcc')
#SRParser('abbcc'),SRParser('ababbcc')


STACK			ISUT			ACTION
['$', 0, 'a', 3]    		bbc$    		SHIFT S3
['$', 0, 'a', 3, 'b', 6]    		bc$    		SHIFT S6
['$', 0, 'A']    		bc$    		REDUCE R3
['$', 0, 'A', 2]    		bc$    		GOTO 2
['$', 0, 'A', 2, 'b', 4]    		c$    		SHIFT S4
['$', 0, 'A', 2, 'B']    		c$    		REDUCE R4
['$', 0, 'A', 2, 'B', 5]    		c$    		GOTO 5
['$', 0, 'A', 2, 'B', 5, 'c', 8]    		$    		SHIFT S8
['$', 0, 'A', 2, 'B', 5, 'C']    		$    		REDUCE R6
['$', 0, 'A', 2, 'B', 5, 'C', 7]    		$    		GOTO 7
['$', 0, 'S']    		$    		REDUCE R1
['$', 0, 'S', 1]    		$    		GOTO 1
['ACC']    		$    		ACCEPT ACC
STACK			ISUT			ACTION
['$', 0, 'a', 3]    		bbbcc$    		SHIFT S3
['$', 0, 'a', 3, 'b', 6]    		bbcc$    		SHIFT S6
['$', 0, 'A']    		bbcc$    		REDUCE R3
['$', 0, 'A', 2]    		bbcc$    		GOTO 2
['$', 0, 'A', 2, 'b', 4]    		bcc$    		SHIFT S4


('Accepted', 'Rejected')

7. YACC program that reads the input expression and convert it to post fix expression. 